## Setup

In [12]:
import json as js

import pandas as pd
import numpy as np
from pprint import pprint
from urllib.parse import urlparse

from utils import get_configs
%matplotlib inline

In [2]:
configvars = get_configs()

## Read datasets

In [3]:

base_df = pd.read_csv(configvars['BaseDataset']['malicious-urls']['og'])
base_df.head()

,URL,Label
0,http://astore.amazon.co.uk/allezvinsfrenchr/de...,Spam
1,http://archive.salisburyjournal.co.uk/2007/3/6...,Spam
2,http://appbasic.jettons.co.uk/links/index.html,Spam
3,http://archive.yorkpress.co.uk/2003/11/6/25684...,Spam
4,http://acard4u.co.uk/product_reviews.php?cPath...,Spam


In [4]:
X, y = base_df['URL'], base_df['Label']

In [6]:
y.value_counts(normalize=True)

Defacement    0.583294
Benign        0.213938
Spam          0.072566
Malware       0.069942
Phishing      0.060260
Name: Label, dtype: float64

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def custom_tokenizer(string):
    final = []
    tokens = [a for a in list(urlparse(string)) if a]
    for t in tokens:
        final.extend(re.compile("[.-]").split(t))
    return final
print (custom_tokenizer('google.com'))
print(custom_tokenizer('https://google-so-not-fake.com?fake=False&seriously=True'))

['google', 'com']
['https', 'google', 'so', 'not', 'fake', 'com', 'fake=False&seriously=True']


In [32]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split

vect = CountVectorizer(tokenizer=custom_tokenizer)
rc = RidgeClassifier()
rc_pipe = Pipeline([('vect', vect), ('model', rc)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
scores = cross_val_score(rc_pipe, X_train, y_train, cv=5)
scores.mean() # not good enough!!

rc_pipe.fit(X_train, y_train)




Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function custom_tokenizer at 0x7f8ff7d04b80>)),
                ('model', RidgeClassifier())])

In [50]:
y_predict = rc_pipe.predict(X_test.values)